In [86]:
import pandas as pd

In [47]:
df = pd.read_csv('novel_story_sep.csv', lineterminator='\n')

In [48]:
df.columns

Index(['itemId', 'title', 'pubDate', 'coverSmallUrl', 'coverLargeUrl',
       'publisher', 'customerReviewRank', 'author', 'isbn', 'link', 'story',
       'story_sep'],
      dtype='object')

# LDA 모델에 들어갈 객체(dictionary, corpus) 만들고 학습시키기

In [49]:
import gensim
gensim.__version__

'3.8.3'

In [50]:
import gensim
from gensim import corpora, models
from gensim.models import CoherenceModel
from gensim.test.utils import common_corpus
from gensim.models.wrappers import LdaMallet

In [51]:
stories = df['story_sep']

In [52]:
data_word = []
for story in stories:
    data = list(str(story).split())
    data_word.append(data)

In [53]:
id2word = corpora.Dictionary(data_word)

In [73]:
id2word.filter_extremes(no_below = 5) # 5회 이하로 등장한 단어는 삭제
texts = data_word
corpus = [id2word.doc2bow(text) for text in texts]

mallet_path = '../mallet-2.0.8/bin/mallet' 
ldamallet = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=12, random_seed=10, id2word=id2word)

coherence_model_ldamallet = CoherenceModel(model=ldamallet, texts=texts, dictionary=id2word, coherence='c_v')
coherence_ldamallet = coherence_model_ldamallet.get_coherence()
print(coherence_ldamallet) #응집성 지수

0.3956724529838395


In [74]:
# Select the model and print the topics
# optimal_model = model_list[model_list_num]
optimal_model = ldamallet
model_topics = optimal_model.show_topics(formatted=False)
print(optimal_model.print_topics()[5])

(5, '0.040*"세계" + 0.022*"한국" + 0.019*"현실" + 0.015*"편의" + 0.015*"의미" + 0.014*"미래" + 0.014*"사회" + 0.013*"인물" + 0.012*"소설가" + 0.012*"젊은"')


In [75]:
def format_topics_sentences(ldamodel=optimal_model, corpus=corpus, texts=texts, Data=df):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    #ldamodel[corpus]: lda_model에 corpus를 넣어 각 토픽 당 확률을 알 수 있음
    for i, row in enumerate(ldamodel[corpus]):
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num,topn=10)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']
    print(type(sent_topics_df))

    # Add original text to the end of the output
    contents = pd.Series(texts)
    # sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    sent_topics_df = pd.concat([Data['itemId'],Data['title'],Data['story_sep'], sent_topics_df], axis=1)

    return(sent_topics_df)

In [76]:
df_topic_sents_keywords = format_topics_sentences(ldamodel=optimal_model, corpus=corpus, texts=texts, Data=df)

<class 'pandas.core.frame.DataFrame'>


In [77]:
# Format
df_topic_story = df_topic_sents_keywords

# 각 문서에 대한 토픽 내림차순
df_topic_sort=df_topic_story.sort_values(by=['Dominant_Topic'])

In [78]:
# Group top 5 sentences under each topic
sent_topics_sorteddf_mallet = pd.DataFrame()

sent_topics_outdf_grpd = df_topic_sents_keywords.groupby('Dominant_Topic')

for i, grp in sent_topics_outdf_grpd:
    sent_topics_sorteddf_mallet = pd.concat([sent_topics_sorteddf_mallet, 
                                             grp.sort_values(['Perc_Contribution'], ascending=[0]).head(1)], 
                                            axis=0)


In [79]:
sent_topics_sorteddf_mallet

,itemId,title,story_sep,Dominant_Topic,Perc_Contribution,Topic_Keywords
544,349598232,토와의 정원,있다는 굉장한 일이구 두운 심연 잠겨 올려다보는 맑고 투명한 달팽이 식당 츠바키 문...,0.0,0.4965,"아이, 가족, 청소년, 엄마, 마음, 소년, 자신, 학교, 친구, 소녀"
512,347177916,뮤직숍,그리움 잡고 있으면 사랑 이루어진다 영국 타임스 미국 워싱턴포스트 올해 흥미로운 음...,1.0,0.6747,"사람, 시간, 세상, 어떤, 어머니, 모두, 가지, 하나, 마음, 하고"
332,321327972,제왕업(상),황제 타고난 여인 패업 둘러싼 야망 배신 운명 마주친 뜨거운 사랑 황제 타고난 여인...,2.0,0.5458,"드라마, 세상, 시작, 운명, 판타지, 주인공, 영화, 여인, 게임, 로맨스"
603,340168530,태백산맥 3: 제1부 한의 모닥불,한반도 분단 전쟁 제대로 그려 오늘 역사 이어주고 태백산맥 전남 여수 주둔 있던 국...,3.0,0.8724,"여성, 전쟁, 마을, 다시, 상황, 명의, 함께, 모두, 위기, nan"
192,354469155,전지적 독자 시점 PART 1: 7,토털 웹소설 현재 진행형 레전드 마침내 단행본 만나는 전지 시점 피아 누적 판매 네...,4.0,0.7429,"시작, 세계, 하나, 시리즈, 문학상, 이번, 제작, 판매, 시점, 이미"
467,352390319,유리알 유희,헤르 헤세 노벨문학상 유리 유희 세기 무수 전쟁 인한 정신 황폐 반발 시대 비판 보...,5.0,0.5805,"세계, 한국, 현실, 편의, 의미, 미래, 사회, 인물, 소설가, 젊은"
508,349876668,프랑켄슈타인,인공 지능 유전공학 인간 복제 최근 이슈 담아내면서도 진정한 고민 하는 독특한 고전...,6.0,0.5163,"인간, 자신, 스릴러, 장르, 사고, 상상력, 새로운, 재미, 심리, 또한"
555,235805806,개선문 2,사랑 도피 사치 아닌 평화 안전 기쁨 축제 불안 시대 리다 민음사 세계문학 전집 개...,7.0,0.7040,"인간, 대표, 인류, 그려, 한국, 냈다, 고양이, 살아가는, 희망, 특히"
668,353777168,기괴한 레스토랑 1: 정원사의 선물,달러 구트 백화점 이을 한국 판타지 돌아왔다 기괴한 요괴 레스토랑 벌어지는 시아 신...,8.0,0.4309,"자신, 남자, 인생, 함께, 이름, 남편, 여자, 아들, 결혼, 생활"
510,349967826,기억하는 소설,사람 간사한 동물 잊어버린다고 봐라 무너진다 너무나 쉽게 너무도 빨리 잊는 우리 기...,9.0,0.4210,"사랑, 우리, 기억, 모습, 지금, 죽음, 존재, 시작, 다시, 아버지"


In [81]:
df_topic_story.value_counts('Dominant_Topic').sort_index()

Dominant_Topic
0.0     128
1.0      82
2.0      81
3.0      63
4.0      28
5.0     119
6.0      88
7.0      58
8.0      96
9.0      82
10.0     48
11.0    127
dtype: int64

In [85]:
topic_4 = df_topic_story[df_topic_story['Dominant_Topic']==5]
topic_4[topic_4['Perc_Contribution'] > 0.3]

,itemId,title,story_sep,Dominant_Topic,Perc_Contribution,Topic_Keywords
35,333164626,키르케,학자 김헌 기자 이다혜 강력 추천 고전 존경 끄덕임으로 읽는 새롭게 발굴 하는 고전...,5.0,0.3356,"세계, 한국, 현실, 편의, 의미, 미래, 사회, 인물, 소설가, 젊은"
57,354229384,방금 떠나온 세계,한국 눈부신 미래 초엽 사랑 끝내 이해 당신 있지 않나요 지금 초엽 먼저 소개 작가...,5.0,0.3179,"세계, 한국, 현실, 편의, 의미, 미래, 사회, 인물, 소설가, 젊은"
280,351027803,스노 크래시 2,타임 뛰어난 영문 서인 인류 미래 변화 시킨 충격 메타 버스 아바타 탄생 시키며 세...,5.0,0.4697,"세계, 한국, 현실, 편의, 의미, 미래, 사회, 인물, 소설가, 젊은"
290,351027793,스노 크래시 1,타임 뛰어난 영문 서인 인류 미래 변화 시킨 충격 메타 버스 아바타 탄생 시키며 세...,5.0,0.4722,"세계, 한국, 현실, 편의, 의미, 미래, 사회, 인물, 소설가, 젊은"
448,254600571,너무 한낮의 연애,젊은 중균 세계 젊은 대상 너무 한낮 연애 편의 성하게 담긴 집은 김금희 세계 확고...,5.0,0.3140,"세계, 한국, 현실, 편의, 의미, 미래, 사회, 인물, 소설가, 젊은"
451,252098561,제7회 젊은작가상 수상작품집(2016),성실하고 활발하게 자신 세계 조해 가는 일곱 명의 젊은 젊은 수상 작품집 제정 젊은...,5.0,0.3479,"세계, 한국, 현실, 편의, 의미, 미래, 사회, 인물, 소설가, 젊은"
467,352390319,유리알 유희,헤르 헤세 노벨문학상 유리 유희 세기 무수 전쟁 인한 정신 황폐 반발 시대 비판 보...,5.0,0.5805,"세계, 한국, 현실, 편의, 의미, 미래, 사회, 인물, 소설가, 젊은"
471,353740895,0%를 향하여,누구 마약 예술 맞아요 청춘 시대 다시 시작 기쁜 우리 젊은 동시 대의 감각 포착 ...,5.0,0.3330,"세계, 한국, 현실, 편의, 의미, 미래, 사회, 인물, 소설가, 젊은"
550,252098561,제7회 젊은작가상 수상작품집(2016),성실하고 활발하게 자신 세계 조해 가는 일곱 명의 젊은 젊은 수상 작품집 제정 젊은...,5.0,0.3555,"세계, 한국, 현실, 편의, 의미, 미래, 사회, 인물, 소설가, 젊은"
688,353740895,0%를 향하여,누구 마약 예술 맞아요 청춘 시대 다시 시작 기쁜 우리 젊은 동시 대의 감각 포착 ...,5.0,0.3369,"세계, 한국, 현실, 편의, 의미, 미래, 사회, 인물, 소설가, 젊은"
